In [ ]:
# https://www.kaggle.com/shubh24/wordnet-similarity-matrix-a-naive-implementation

In [8]:
import pandas as pd

from nltk.corpus import wordnet as wn

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

df_train = pd.read_csv('data/train.csv')

In [9]:
def get_terms(sentence):
    return [i for i in sentence.lower().split() if i not in stop]

In [10]:
row = df_train.iloc[0] #Just taking the first row, you can put a loopover 

res = row["is_duplicate"]
terms1 = get_terms(row["question1"])
terms2 = get_terms(row["question2"])

sims = []

In [11]:
for word1 in terms1:
    word1_sim = []

    try:
        syn1 = wn.synsets(word1)[0]
    except:  #if wordnet is not able to find a synset for word1
        sims.append([0 for i in range(0, len(terms2))])
        continue


    for word2 in terms2:
        try:
            syn2 = wn.synsets(word2)[0]
        except: #if wordnet is not able to find a synset for word2
            word1_sim.append(0)
            continue

        word_similarity = syn1.wup_similarity(syn2)
        word1_sim.append(word_similarity)

    sims.append(word1_sim)

In [12]:
word1_score = 0
for i in range(0, len(terms1), 1):
    try:
        word1_score += max(sims[i])
    except:
        continue
word1_score /= len(terms1) #Averaging over all terms

In [13]:
word2_score = 0

for i in range(0, len(terms2), 1):
    try:
        word2_score += max([j[i] for j in sims])
    except:
        continue
word2_score /= len(terms2)

In [15]:
pair_score = (word1_score + word2_score)/2
pair_score

0.40476190476190477